In [0]:
! pip install wosplus openpyxl >/dev/null

In [2]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oadoi_udea.json         = 1s508qokcIkz-i1Y4Ukbdht-jWuSVngON

Writing drive.cfg


In [0]:
import wosplus as wp
files=wp.wosplus('drive.cfg')

In [0]:
import pandas as pd

In [0]:
df=files.read_drive_json('oadoi_udea.json')

In [0]:
dfin=df[df.best_oa_location.isnull()].reset_index(drop=True)
dfinn=df[~df.best_oa_location.isnull()].reset_index(drop=True)
dfinn['url_for_pdf']=dfinn.best_oa_location.apply(lambda x: x.get('url_for_pdf'))
dfin['url_for_pdf']=None
dff=dfinn.append(dfin).reset_index(drop=True)

In [0]:
dff=dff.fillna('')

In [0]:
pdfs=dff[dff['url_for_pdf']!=''] # dejar solo los pdfs
pdfs=pdfs[pdfs['z_authors']!=''] # dejar solo los pdfs con authors

In [155]:
pdfs.shape

(2651, 18)

In [0]:
pdfs.to_json('oadoi.json')

In [0]:
pdfs.to_json('oadoi.jsonl',orient='records',lines=True)

In [0]:
pdfs=pdfs.reset_index(drop=True)

In [0]:
pdfs['autor_1']=pdfs.z_authors.str[0].apply(lambda x: x.get('family') )+', '+pdfs.z_authors.str[0].apply(lambda x: x.get('given') )

In [0]:
def isd(x):
  if type(x)==str:
    return eval(x)
  else:
    return x

In [0]:
pdfs['autor_2']=pdfs.z_authors.str[1].fillna('{}').apply(isd).apply(lambda x: x.get('family') )+', '+pdfs.z_authors.str[1].fillna('{}').apply(isd).apply(lambda x: x.get('family') )

In [148]:
pdfs.columns

Index(['best_oa_location', 'data_standard', 'doi', 'doi_url', 'genre', 'is_oa',
       'journal_is_in_doaj', 'journal_is_oa', 'journal_issns', 'journal_name',
       'oa_locations', 'published_date', 'publisher', 'title', 'updated',
       'year', 'z_authors', 'url_for_pdf', 'autor_1', 'autor_2'],
      dtype='object')

In [0]:
pdfs[['doi','title','autor_1', 'autor_2', 'url_for_pdf']].to_excel('investigacion.xlsx',index=False)